## Basic Setup

Run the cells below for the basic setup of this notebook.

In [1]:
try:
    from google.colab import drive # type: ignore
    IN_COLAB = True
except:
    IN_COLAB = False
    print('No colab environment, assuming local setup.')

if IN_COLAB:
    drive.mount('/content/drive')

    # TODO: Enter the foldername in your Drive where you have saved the unzipped
    # turorials folder, e.g. 'alphafold-decoded/tutorials'
    FOLDERNAME = None
    assert FOLDERNAME is not None, "[!] Enter the foldername."

    # Now that we've mounted your Drive, this ensures that
    # the Python interpreter of the Colab VM can load
    # python files from within it.
    import sys
    sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
    %cd /content/drive/My\ Drive/$FOLDERNAME

    print('Connected COLAB to Google Drive.')

import os
    
base_folder = '../feature_embedding'
control_folder = f'{base_folder}/control_values'

assert os.path.isdir(control_folder), 'Folder "control_values" not found, make sure that FOLDERNAME is set correctly.' if IN_COLAB else 'Folder "control_values" not found, make sure that your root folder is set correctly.'

No colab environment, assuming local setup.


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from matplotlib import pyplot as plt
import torch
import math

In [4]:
torch.set_grad_enabled(False)

# Feature Embedding

In this notebook, we are going to implement Alphafold's feature embedding pipeline. The process of the pipeline is visualized in the diagram below. So far, we extracted `extra_msa_feat`, `residue_index`, `target_feat` and `msa_feat` from the sequence alignment data during feature extraction. The embedding of these features is done in three steps:
1. `InputEmbedder`: The steps in the diagram below that produce the pair representation and the MSA representation.

2. `RecyclingEmbedder`: Denoted with 'R' in the diagram below. Alphafold is run multiple times, and the transformed pair and MSA representations, as well as the predicted atom positions, are fed back into the new iterations via the RecyclingEmbedder.

3. `ExtraMsaStack`: The extra MSA stack feeds in additional data from the sequences in the alignment that were not selected as cluster centers. It is architecturally similar to the Evoformer, and we can reuse almost all primitives from it.

Optionally, Alphafold can also use features from tempmlate protein structures. We aren't implementing this feature, that's why this part of the diagram is grayed out.

<figure align=center style="padding: 30px">
<img src='images/alphafold_embedding_pipeline_grayed.png'>
<figcaption>Source: Jumper, J., Evans, R., Pritzel, A. et al. Highly accurate protein structure prediction with AlphaFold. Nature 596, 583–589 (2021).</figcaption>
</figure>

## Input Embedder

We'll start right away by implementing the input embedder. It is described in Algorithm 3 and Algorithm 4 in the paper's supplement. Note that in Algorithm 3 Line 1, even though one linear module is written, two separate modules `linear_tf_z_i` and `linear_tf_z_j` are used for the construction of `a` and `b` respectively.

Head over to `input_embedder.py`, implement the `__init__` method and check your implementation by running the following cell.

In [8]:
from pathlib import Path 
import sys
base_folder = Path('/Users/thomasbush/Documents/ML/alphafold-decoded/tutorials')

# Remove wrong paths if any
sys.path = [p for p in sys.path if 'feature_embedding' not in p]

# Add base path again
sys.path.insert(0, str(base_folder))

In [15]:
from feature_embedding.control_values.embedding_checks import test_module_shape
from feature_embedding.control_values.embedding_checks import c_m, c_z, tf_dim, msa_feat_dim
from feature_embedding.input_embedder import InputEmbedder

input_embedder = InputEmbedder(c_m, c_z, tf_dim, msa_feat_dim=msa_feat_dim, vbins=32)

test_module_shape(input_embedder, 'input_embedder', control_folder)

As you can see in the diagram, the residue index is processed by `relpos` and added to the pair representation. For this, the residue index of shape (N_res,) (its value is [0,...,N_res-1]) needs to match the shape of the pair representation, which is (N_res, N_res, c_z). To get the shape (N_res, N_res), the outer difference of the residue index with itself is calculated. This results in the matrix
$$\begin{pmatrix} 0 & -1 & -2 & -3 & ... \\ 1 & 0 & -1 & -2 & ...\\ 2 & 1 & 0 & -1 & ...\\ 3 & 2 & 1 & 0 & ... \end{pmatrix}$$
of shape (N_res, N_res). The values are clamped to the range (-32, 32) and one-hot encoded, resulting in a tensor of shape (N_res, N_res, 2*32+1). Finally, this tensor is passed through a linear layer and gets to the shape (N_res, N_res, c_z).

All of this is done in `relpos`. Implement the method and run the following cell to check your code.

In [22]:
from feature_embedding.control_values.embedding_checks import test_module_method, c_m, c_z, tf_dim, msa_feat_dim

input_embedder = InputEmbedder(c_m, c_z, tf_dim, msa_feat_dim=msa_feat_dim, vbins=32)

test_module_method(input_embedder, 'input_embedder_relpos', 'residue_index', 'z_out', control_folder, lambda x: input_embedder.relpos(x))

/Users/thomasbush/Documents/ML/alphafold-decoded/tutorials/feature_embedding/control_values/embedding_checks.py:133: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  expected_o

`relpos` is used in the forward pass of the input embedder to embed the residue indices. Now, implement the forward pass of the input embedder according to Algorithm 3 and check your code with the following cell.

In [26]:
from feature_embedding.control_values.embedding_checks import test_module_forward, c_m, c_z, tf_dim, msa_feat_dim

input_embedder = InputEmbedder(c_m, c_z, tf_dim, msa_feat_dim=msa_feat_dim, vbins=32)

test_module_forward(input_embedder, 'input_embedder', 'batch', ('m_out', 'z_out'), control_folder)

RuntimeError: The size of tensor a (5) must match the size of tensor b (35) at non-singleton dimension 1

## Recycling Embedder

Alphafold is run multiple times, and the outputs of the last iteration are fed into the next one via the recycling embedder. Concretely, the first line of the MSA representation, the pair representation and the pseudo-beta positions are used. The pseudo-beta positions are the 3-dimensional coordinates of the C-beta atoms of each amino acid in the protein, as computed by the Structure Module in the last iteration. They are termed 'pseudo-beta' as the C-alpha is used for glycine. But this is something we will worry about when computing these pseudo-beta positions in the Structure Module.

The Recycling Embedder is described in Algorithm 32. The implementation in the AlphaFold codebase differs a little from the described algorithm. The file `recycling_embedder.py` contains step-by-step instructions for you. Implement the init method and the forward pass, then check your code by running the following cell.

In [8]:
from feature_embedding.control_values.embedding_checks import c_m, c_z, test_module_shape, test_module_forward
from feature_embedding.recycling_embedder import RecyclingEmbedder

recycling_embedder = RecyclingEmbedder(c_m, c_z)

test_module_shape(recycling_embedder, 'recycling_embedder', control_folder)

test_module_forward(recycling_embedder, 'recycling_embedder', ('m', 'z', 'x'), ('m_out', 'z_out'), control_folder)

## Extra MSA Stack

During feature extraction, some of the sequences are randomly selected as cluster centers, and the non-selected sequences only contribute as cluster averages to the MSA feature. To incorporate additional information from these non-selected sequences, the Extra MSA Stack processes a large selection of these sequences in a less memory-intensive fashion, using so called 'global self-attention'. 

The Extra MSA pipeline starts of simple with a linear embedding. This is done in `ExtraMsaEmbedder` in the file `extra_msa_stack.py`. Implement the module and check your code by running the following cell.

In [9]:
from feature_embedding.control_values.embedding_checks import test_module_shape, test_module_forward, f_e, c_e
from feature_embedding.extra_msa_stack import ExtraMsaEmbedder

extra_msa_embedder = ExtraMsaEmbedder(f_e, c_e)

test_module_shape(extra_msa_embedder, 'extra_msa_embedder', control_folder)

test_module_forward(extra_msa_embedder, 'extra_msa_embedder', 'batch', 'e_out', control_folder)

This embedding is passed through the ExtraMSAStack. It is described in Algorithm 18. It is similar to the EvoformerStack, the main difference being the use of global self-attention instead of normal self-attention. Global self-attention is only using one, averaged query vector. This drastically reduced the memory consumption from $O(n^2)$ to $O(n)$, but also reduces the complexity of the outputs: The attention mechanism just generates one output vector, and it's only brought back to `N_extra` by being broadcasted during gating in the attention module.

We already implemented global self-attention in our attention module, so the implementation of `MSAColumnGlobalAttention` is mostly about setting the right parameters. Check your code with the following cell.

In [10]:
from feature_embedding.control_values.embedding_checks import test_module_shape, test_module_forward, c_m, c_z,c,N_head
from feature_embedding.extra_msa_stack import MSAColumnGlobalAttention

msa_global_col_att = MSAColumnGlobalAttention(c_m, c_z, c, N_head)

test_module_shape(msa_global_col_att, 'msa_global_col_att', control_folder)

test_module_forward(msa_global_col_att, 'msa_global_col_att', 'm', 'm_out', control_folder)

The ExtraMSAStack is, just like the Evoformer, build up from a number of identical blocks. It is described in Algorithm 18, and the differences to the evoformer stack are highlighted in yellow. Implement the module `ExtraMSABlock` and check your code by running the following cell.

In [11]:
from feature_embedding.control_values.embedding_checks import test_module_shape, test_module_forward, c_e, c_z
from feature_embedding.extra_msa_stack import ExtraMsaBlock

extra_msa_block = ExtraMsaBlock(c_e, c_z)

test_module_shape(extra_msa_block, 'extra_msa_block', control_folder)

test_module_forward(extra_msa_block, 'extra_msa_block', ('e', 'z'), 'm_out', control_folder)

Last up is `ExtraMsaStack`, which wraps the extra MSA blocks and executes them sequentially. Implement it and check your code with the following cell.

In [12]:
from feature_embedding.control_values.embedding_checks import test_module_shape, test_module_forward, c_e, c_z
from feature_embedding.extra_msa_stack import ExtraMsaStack

num_blocks = 3
extra_msa_stack = ExtraMsaStack(c_e, c_z, num_blocks)

test_module_shape(extra_msa_stack, 'extra_msa_stack', control_folder)

test_module_forward(extra_msa_stack, 'extra_msa_stack', ('e', 'z'), 'm_out', control_folder)

## Conclusion

With that, we are finally done with the full model up to the evoformer. In theory, we are only missing the structure module now. However, implementing the Structure Module requires quite a bit of knowledge about 3D geometry: Rotations, quaternions, coordinate frames and homogenous coordinates. To get you covered, we put in a notebook on these concepts. See you there!